In [18]:
import torch
import logging
import torch.nn as nn
from tqdm import tqdm
from pathlib import Path
from torch import autograd
from sklearn import model_selection
from torch.utils.data import Dataset, Subset, DataLoader

from strokescopy import StrokePatientsMIDataset, StrokePatientsMIProcessedDataset
from strokesdict import STROKEPATIENTSMI_LOCATION_DICT
import scipy
from torcheeg.transforms import Select,BandSignal,Compose
from to import ToGrid, ToTensor
from typing import Callable, Dict, Union, List
import numpy as np
import soxr
from downsample import SetSamplingRate
from baseline import BaselineCorrection
from torcheeg.transforms import EEGTransform, Select,BandSignal,Compose

dataset = StrokePatientsMIDataset(root_path='../../../../fm/datasets/stroke_data',
                                #   io_path='.torcheeg/datasets_1741414835278_qQ4e6',
                        chunk_size=500,  # 1 second
                        overlap = 250,
                        offline_transform=Compose(
                                [BaselineCorrection(),
                                SetSamplingRate(origin_sampling_rate=500,target_sampling_rate=128),
                                BandSignal(sampling_rate=128,band_dict={'frequency_range':[8,40]})
                                ]),
                        online_transform=Compose(
                                # [ToTensor()]),
                                [ToGrid(STROKEPATIENTSMI_LOCATION_DICT),ToTensor()]),
                
                        label_transform=Select('label'),
                        num_worker=8
)
print(dataset[0][0].shape) #EEG shape:torch.Size([4, 128, 9, 9])
print(dataset[0][1])  # label (int)
print(len(dataset))

[2025-03-09 20:56:41] INFO (torcheeg/MainThread) 🔍 | Processing EEG data. Processed EEG data has been cached to .torcheeg/datasets_1741525001097_DQR1n.
[2025-03-09 20:56:41] INFO (torcheeg/MainThread) ⏳ | Monitoring the detailed processing of a record for debugging. The processing of other records will only be reported in percentage to keep it clean.
[PROCESS]:   0%|          | 0/50 [00:00<?, ?it/s]

[PROCESS]:  16%|█▌        | 8/50 [00:00<00:03, 11.25it/s]
[RECORD ../../../../fm/datasets/stroke_data/sourcedata/sub-01/sub-01_task-motor-imagery_eeg.mat]: 0it [00:00, ?it/s]
[RECORD ../../../../fm/datasets/stroke_data/sourcedata/sub-01/sub-01_task-motor-imagery_eeg.mat]: 1it [00:00,  2.19it/s]
[RECORD ../../../../fm/datasets/stroke_data/sourcedata/sub-01/sub-01_task-motor-imagery_eeg.mat]: 2it [00:00,  3.27it/s]
[RECORD ../../../../fm/datasets/stroke_data/sourcedata/sub-01/sub-01_task-motor-imagery_eeg.mat]: 10it [00:00, 17.51it/s]
[RECORD ../../../../fm/datasets/stroke_data/sourcedata/sub-01/sub-01_task-motor-imagery_eeg.mat]: 13it [00:00, 19.94it/s]
[RECORD ../../../../fm/datasets/stroke_data/sourcedata/sub-01/sub-01_task-motor-imagery_eeg.mat]: 16it [00:01, 21.92it/s]
[RECORD ../../../../fm/datasets/stroke_data/sourcedata/sub-01/sub-01_task-motor-imagery_eeg.mat]: 19it [00:01, 23.72it/s]
[RECORD ../../../../fm/datasets/stroke_data/sourcedata/sub-01/sub-01_task-motor-imagery_eeg.mat

torch.Size([1, 128, 9, 9])
0
12000


In [144]:
class Generator(nn.Module):
    def __init__(self, in_channels=128, out_channels=128):
        super(Generator, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels,
                      128,
                      kernel_size=3,
                      stride=1,
                      padding=1,
                      bias=True), nn.LeakyReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=5, stride=1, padding=2, bias=True),
            nn.LeakyReLU())
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 32, kernel_size=5, stride=1, padding=2, bias=True),
            nn.LeakyReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(32, 16, kernel_size=3, stride=1, padding=1, bias=True),
            nn.LeakyReLU())
        self.delayer1 = nn.Sequential(
            nn.ConvTranspose2d(16 + 32,
                               32,
                               kernel_size=3,
                               stride=1,
                               padding=1,
                               bias=True), nn.LeakyReLU())
        self.delayer2 = nn.Sequential(
            nn.ConvTranspose2d(32 + 64,
                               64,
                               kernel_size=3,
                               stride=1,
                               padding=1,
                               bias=True), nn.LeakyReLU())
        self.delayer3 = nn.Sequential(
            nn.ConvTranspose2d(64 + 128,
                               128,
                               kernel_size=3,
                               stride=1,
                               padding=1,
                               bias=True))

    def forward(self, x):
        #         x = channel_to_location(x)
        x = x.squeeze(1)
        mask = (x.abs().sum(dim=1, keepdim=True) > 0).float()
        out1 = self.layer1(x)
        out2 = self.layer2(out1)
        out3 = self.layer3(out2)
        out = self.layer4(out3)
        out = self.delayer1(torch.cat([out, out3], dim=1))
        out = self.delayer2(torch.cat([out, out2], dim=1))
        out = self.delayer3(torch.cat([out, out1], dim=1))
        out = out * mask
        return out.unsqueeze(1)

In [19]:
class Generator(nn.Module):
    def __init__(self, in_channels=128, out_channels=128):
        super(Generator, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels,
                      128,
                      kernel_size=3,
                      stride=1,
                      padding=1,
                      bias=True), nn.LeakyReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=5, stride=1, padding=2, bias=True),
            nn.LeakyReLU())
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 32, kernel_size=5, stride=1, padding=2, bias=True),
            nn.LeakyReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(32, 16, kernel_size=3, stride=1, padding=1, bias=True),
            nn.LeakyReLU())
        self.delayer1 = nn.Sequential(
            nn.ConvTranspose2d(16 + 32,
                               32,
                               kernel_size=3,
                               stride=1,
                               padding=1,
                               bias=True), nn.LeakyReLU())
        self.delayer2 = nn.Sequential(
            nn.ConvTranspose2d(32 + 64,
                               64,
                               kernel_size=3,
                               stride=1,
                               padding=1,
                               bias=True), nn.LeakyReLU())
        self.delayer3 = nn.Sequential(
            nn.ConvTranspose2d(64 + 128,
                               128,
                               kernel_size=3,
                               stride=1,
                               padding=1,
                               bias=True))

    def forward(self, x):
        #         x = channel_to_location(x)
        x = x.squeeze(1)
        mask = (x.abs().sum(dim=1, keepdim=True) > 0).float()

        # Save the input signal's mean and std to apply after generation
        input_mean = x.mean(dim=[2, 3], keepdim=True)
        input_std = x.std(dim=[2, 3], keepdim=True)

        out1 = self.layer1(x)
        out2 = self.layer2(out1)
        out3 = self.layer3(out2)
        out = self.layer4(out3)
        out = self.delayer1(torch.cat([out, out3], dim=1))
        out = self.delayer2(torch.cat([out, out2], dim=1))
        out = self.delayer3(torch.cat([out, out1], dim=1))

        # Normalize the output to match the input's mean and std
        out = (out - out.mean(dim=[2, 3], keepdim=True)) / (out.std(dim=[2, 3], keepdim=True) + 1e-6)
        out = out * input_std + input_mean
        
        out = out * mask
        return out.unsqueeze(1)

In [183]:
g_model = Generator()

# 模拟输入
input_tensor = torch.randn(2,1, 128, 9, 9)  # [batch, 1, 128, 7, 5]
output = g_model(input_tensor)

print("Output shape:", output.shape)



Output shape: torch.Size([2, 1, 128, 9, 9])


In [2]:
import torch
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super(Generator, self).__init__()
        
        # 3D 卷积处理时间 (128) + 空间 (9x9)
        self.layer1 = nn.Sequential(
            nn.Conv3d(in_channels, 128, kernel_size=(3, 3, 3), stride=1, padding=1, bias=True), 
            nn.LeakyReLU()
        )
        self.layer2 = nn.Sequential(
            nn.Conv3d(128, 64, kernel_size=(3, 3, 3), stride=1, padding=1, bias=True), 
            nn.LeakyReLU()
        )
        self.layer3 = nn.Sequential(
            nn.Conv3d(64, 32, kernel_size=(3, 3, 3), stride=1, padding=1, bias=True), 
            nn.LeakyReLU()
        )
        self.layer4 = nn.Sequential(
            nn.Conv3d(32, 16, kernel_size=(3, 3, 3), stride=1, padding=1, bias=True), 
            nn.LeakyReLU()
        )

        # 反卷积上采样
        self.delayer1 = nn.Sequential(
            nn.ConvTranspose3d(16 + 32, 32, kernel_size=(3, 3, 3), stride=1, padding=1, bias=True), 
            nn.LeakyReLU()
        )
        self.delayer2 = nn.Sequential(
            nn.ConvTranspose3d(32 + 64, 64, kernel_size=(3, 3, 3), stride=1, padding=1, bias=True), 
            nn.LeakyReLU()
        )
        self.delayer3 = nn.Sequential(
            nn.ConvTranspose3d(64 + 128, 128, kernel_size=(3, 3, 3), stride=1, padding=1, bias=True), 
            nn.LeakyReLU()
        )

        # 最终输出通道数调整回原始输入的 4（频带数）
        self.final_layer = nn.Conv3d(128, out_channels, kernel_size=(3, 3, 3), stride=1, padding=1, bias=True)

    def forward(self, x):
        """
        输入: x (batch, 4, 128, 9, 9)
        """
        mean, std = x.mean(), x.std()
        mask = (x.abs().sum(dim=1, keepdim=True) > 0).float()  # 计算 mask (batch, 1, 128, 9, 9)
        
        # 编码器
        out1 = self.layer1(x)  # (batch, 128, 128, 9, 9)
        out2 = self.layer2(out1)  # (batch, 64, 128, 9, 9)
        out3 = self.layer3(out2)  # (batch, 32, 128, 9, 9)
        out = self.layer4(out3)  # (batch, 16, 128, 9, 9)

        # 解码器（跳跃连接）
        out = self.delayer1(torch.cat([out, out3], dim=1))  # (batch, 32, 128, 9, 9)
        out = self.delayer2(torch.cat([out, out2], dim=1))  # (batch, 64, 128, 9, 9)
        out = self.delayer3(torch.cat([out, out1], dim=1))  # (batch, 128, 128, 9, 9)

        # 还原到原始通道数 4
        out = self.final_layer(out)  # (batch, 4, 128, 9, 9)
        
        output = out * std + mean
        return output*mask  # 保留 mask 约束



In [30]:
def random_mask(data, min_r=0.0, max_r=0.2):
    """
    对 EEG 数据进行随机掩码操作，使部分时间点的数据置零。

    参数:
    - data: (batch_size, 4, 128, 9, 9) 的 EEG 数据
    - min_r: 掩码最小比例
    - max_r: 掩码最大比例

    返回:
    - masked_data: 掩码后的 EEG 数据
    - ratio: 采样的掩码比例
    """
    data = data.clone()
    
    # 生成随机 mask，形状 (batch_size, 4, 128, 1, 1)
    mask = torch.rand(*data.shape[:3], *([1] * (len(data.shape) - 3)), device=data.device)

    # 生成随机掩码阈值 (batch_size, 1, 1, 1, 1)，然后广播
    ratio = torch.rand(size=(data.shape[0], 1, 1, 1, 1), device=data.device) * (max_r - min_r) + min_r

    # 生成掩码矩阵 (batch_size, 4, 128, 9, 9)
    mask = mask < ratio
    mask = mask.expand_as(data)

    # 置零
    data[mask] = 0.0

    return data, ratio


In [20]:
def random_mask(data, min_r=0.0, max_r=0.4):
    """
    对单通道 EEG 数据进行随机掩码操作，使部分时间点的数据置零。

    参数:
    - data: (batch_size, 1, 128, 9, 9) 的 EEG 数据
    - min_r: 掩码最小比例
    - max_r: 掩码最大比例

    返回:
    - masked_data: 掩码后的 EEG 数据
    - ratio: 采样的掩码比例
    """
    data = data.clone()
    
    # **修正：mask 形状调整为单通道**
    mask = torch.rand(data.shape[0], 1, data.shape[2], 1, 1, device=data.device)

    # 生成随机掩码阈值 (batch_size, 1, 1, 1, 1)，然后广播
    ratio = torch.rand(size=(data.shape[0], 1, 1, 1, 1), device=data.device) * (max_r - min_r) + min_r

    # 生成掩码矩阵 (batch_size, 1, 128, 9, 9)
    mask = mask < ratio
    mask = mask.expand_as(data)

    # 置零
    data[mask] = 0.0

    return data, ratio

In [4]:
from eegswintransformer import SwinTransformer

d_model = SwinTransformer(patch_size=(4,3,3),
                        num_classes=1,
                        depths=(2, 4, 4),
                        num_heads=(2,3,4),
                        window_size=(3,3,3),
                        in_chans=1,
                        drop_rate=0.2,
                        attn_drop_rate=0.2,
                        drop_path_rate=0.3
                        ) # T, W, H 同时缩小
g_model = Generator()

/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [4]:
import torch
import numpy as np
import torch.nn as nn

from tqdm import tqdm
from pathlib import Path
from torch import autograd
from sklearn import model_selection
from torch.utils.data import Dataset, Subset, DataLoader


RECEIVED_PARAMS = {
    "c_lr": 0.00001,
    "g_lr": 0.00001,
    "d_lr": 0.00001,
    "weight_gp": 1.0,
    "weight_decay": 0.0005,
    "weight_ssl": 0.5
}
TRAIL_ID = 'cross_validation_proposed_pretrain'


class CFG:
    NUM_EPOCHS = 100
    NUM_CLASSES = 2
    BATCH_SIZE = 16
    TIMESTEP_NUM = 128


def train_test_split(dataset, test_size=0.2, random_state=520, shuffle=True):
    n_samples = len(dataset)
    indices = np.arange(n_samples)
    train_index, test_index = model_selection.train_test_split(
        indices,
        test_size=test_size,
        random_state=random_state,
        shuffle=shuffle)

    train_dataset = Subset(dataset, train_index)
    test_dataset = Subset(dataset, test_index)

    return train_dataset, test_dataset


def gradient_penalty(model, real, fake):
    device = real.device
    real = real.data
    fake = fake.data
    alpha = torch.rand(real.size(0), *([1] * (len(real.shape) - 1))).to(device)
    inputs = alpha * real + ((1 - alpha) * fake)
    inputs.requires_grad_()
    outputs = model(inputs)

    gradient = autograd.grad(outputs=outputs,
                             inputs=inputs,
                             grad_outputs=torch.ones_like(outputs).to(device),
                             create_graph=True,
                             retain_graph=True,
                             only_inputs=True)[0]

    gradient = gradient.flatten(1)
    return ((gradient.norm(2, dim=1) - 1)**2).mean()


class Trainer():
    def __init__(self, g_model, d_model, trainer_kwargs={'max_epochs': 10}):
        super().__init__()
        self.g_model = g_model.cuda()
        self.d_model = d_model.cuda()

        self._loss_fn_ce = nn.CrossEntropyLoss()
        self._loss_fn_mse = nn.MSELoss()
        self._optimizer_g_model = torch.optim.Adam(
            g_model.parameters(),
            lr=RECEIVED_PARAMS['g_lr'],
            weight_decay=RECEIVED_PARAMS['weight_decay'])
        self._optimizer_d_model = torch.optim.Adam(
            d_model.parameters(),
            lr=RECEIVED_PARAMS['d_lr'],
            weight_decay=RECEIVED_PARAMS['weight_decay'])

        self._trainer_kwargs = trainer_kwargs

        eeg_dataset = dataset
        train_dataset, val_dataset = train_test_split(eeg_dataset)
        train_dataloader = DataLoader(train_dataset,
                                      batch_size=CFG.BATCH_SIZE,
                                      shuffle=True,
                                      drop_last=False)

        self._train_dataloader = train_dataloader

    def _accuracy(self, input, target):  # pylint: disable=redefined-builtin
        _, predict = torch.max(input.data, 1)
        correct = predict.eq(target.data).cpu().sum().item()
        return correct / input.size(0)

    def training_step_g_model(self, batch, batch_idx, augment_fn=random_mask):
        self._optimizer_g_model.zero_grad()

        for p in self.d_model.parameters():
            p.requires_grad = False

        x, y = batch
        x, y = x.cuda(), y.cuda()

        aug_x, ratio = random_mask(x)
        pred_x = self.g_model(aug_x)
        loss = -self.d_model(pred_x).mean() # 生成数据输入到d_model得到的数值越高，就越真。加个负号即最大化该数值

        loss.backward()
        self._optimizer_g_model.step()

        return loss

    def training_step_d_model(self, batch, batch_idx, augment_fn=random_mask):
        self._optimizer_d_model.zero_grad()

        for p in self.d_model.parameters():
            p.requires_grad = True

        x, y = batch
        x, y = x.cuda(), y.cuda()

        aug_x, ratio = random_mask(x)
        pred_x = self.g_model(aug_x).detach()

        fake_score = self.d_model(pred_x).mean()
        real_score = self.d_model(x).mean()
        loss = fake_score - real_score # 最大化生成数据和真实数据的差异
        # print("fake:",fake_score," real:",real_score)
        loss += RECEIVED_PARAMS['weight_gp'] * gradient_penalty(
            self.d_model, x, pred_x)

        if batch_idx % 5 == 0:
            loss.backward()
            self._optimizer_d_model.step()

        return loss

    def _train(self, epoch_idx=-1):
        """
        单独显示每个 epoch 的训练进度条，并动态更新 G 和 D 的损失。
        """
        pbar = tqdm(total=len(self._train_dataloader), desc=f"[TRAIN] Epoch {epoch_idx}")
        for i, batch in enumerate(self._train_dataloader):
            # 获取 D 模型的损失
            loss_d_model = self.training_step_d_model(batch, i)
            # 获取 G 模型的损失
            loss_g_model = self.training_step_g_model(batch, i)

            # 更新进度条
            pbar.update(1)
            pbar.set_postfix(
                ordered_dict={
                    'loss_g_model': f'{loss_g_model.item():.3f}',
                    'loss_d_model': f'{loss_d_model.item():.3f}'
                }
            )
        pbar.close()
        
    def fit(self) -> None:
        for i in range(self._trainer_kwargs['max_epochs']):
            self._train(i + 1)

    def save(self, param_path):
        torch.save(
            {
                'g_model': self.g_model.state_dict(),
                'd_model': self.d_model.state_dict()
            }, param_path)

In [8]:
trainer = Trainer(g_model,
                  d_model,
                  trainer_kwargs={'max_epochs': CFG.NUM_EPOCHS})
trainer.fit()
trainer.save('./parameters/' + 'cross_validation_proposed_pretrain.pth')

[TRAIN] Epoch 100: 100%|██████████| 14/14 [00:03<00:00,  4.38it/s, loss_g_model=-8.992, loss_d_model=-4.024]


In [79]:
g_model = Generator()
checkpoint = torch.load('./parameters/cross_validation_proposed_pretrain.pth')

# 加载到各自的模型
g_model.load_state_dict(checkpoint['g_model'])
# d_model.load_state_dict(checkpoint['d_model'])


RuntimeError: Error(s) in loading state_dict for Generator:
	size mismatch for layer1.0.weight: copying a param with shape torch.Size([16, 1, 3, 3, 3]) from checkpoint, the shape in current model is torch.Size([128, 128, 3, 3]).
	size mismatch for layer1.0.bias: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([128]).
	size mismatch for layer2.0.weight: copying a param with shape torch.Size([32, 16, 3, 3, 3]) from checkpoint, the shape in current model is torch.Size([64, 128, 5, 5]).
	size mismatch for layer2.0.bias: copying a param with shape torch.Size([32]) from checkpoint, the shape in current model is torch.Size([64]).
	size mismatch for layer3.0.weight: copying a param with shape torch.Size([64, 32, 3, 3, 3]) from checkpoint, the shape in current model is torch.Size([32, 64, 5, 5]).
	size mismatch for layer3.0.bias: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([32]).
	siz

In [6]:
import torch
from torch.utils.data import DataLoader
# 创建随机输入 (batch_size=2, channels=4, time=128, height=9, width=9)
# x = torch.randn(2, 4, 128, 9, 9)
# 取出一个 batch
k_fold = KFoldPerSubjectGroupbyTrial(
    n_splits=4,
    shuffle=True,
    random_state=42)

for i, (training_dataset, test_dataset) in enumerate(k_fold.split(dataset)):
    if i==3:
        x_loader = DataLoader(training_dataset,
                                batch_size=HYPERPARAMETERS['batch_size'],
                                shuffle=True)
        test_loader = DataLoader(test_dataset,
                                batch_size=HYPERPARAMETERS['batch_size'],
                                shuffle=False)
        # GAN 训练 generator & discriminator
        g_model = Generator()
        checkpoint = torch.load('./parameters/cross_validation_proposed_pretrain.pth')
        # 加载到各自的模型
        g_model.load_state_dict(checkpoint['g_model'])
        x_iter = iter(x_loader)  # 创建迭代器
        x_batch = next(x_iter)   # 获取第一个 batch

        x,label = x_batch
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        # 切换到评估模式
        g_model = g_model.to(device)
        g_model.eval()
        with torch.no_grad():
            x = x.to(device)  # 确保输入数据在正确的设备上
            fake = g_model(x)  # 使用生成器进行预测


        print("Output shape:", fake.shape)

        import matplotlib.pyplot as plt

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        # 选择 batch 内的第一个样本
        sample_idx = 8  # 选择第几个样本
        freq_band = 0   # 选择第几个频带
        channel_x = 2   # 选择某个网格坐标 (x, y)
        channel_y = 5
        fake = g_model(x)  # 使用生成器进行预测
        # 获取输入 EEG 数据
        input_eeg = x[sample_idx, freq_band, :, channel_x, channel_y].cpu().detach().numpy()
        output_eeg = fake[sample_idx, freq_band, :, channel_x, channel_y].cpu().detach().numpy()
        # print(input_eeg)
        # 绘制 EEG 时间序列
        plt.figure(figsize=(10, 4))
        plt.plot(input_eeg, label="real EEG", linestyle='-', color='b')
        plt.plot(output_eeg, label="fake EEG", linestyle='--', color='r')
        plt.xlabel("times")
        plt.ylabel("altitude")
        plt.title("EEG real vs. fake")
        plt.legend()
        plt.show()

NameError: name 'KFoldPerSubjectGroupbyTrial' is not defined

In [23]:
c_model = SwinTransformer(patch_size=(4,3,3),
                            num_classes=2,
                            depths=(2, 4, 4),
                            num_heads=(2,3,4),
                            window_size=(3,3,3),
                            in_chans=1,
                            # drop_rate=0.2,
                            # attn_drop_rate=0.2,
                            # drop_path_rate=0.3,
                            visual=True
                            ) # T, W, H 同时缩小

In [6]:
class CTrainer():
    def __init__(self, c_model, trainer_kwargs={'max_epochs': 10}):
        super().__init__()
        self.c_model = c_model.cuda()

        self._loss_fn_ce = nn.CrossEntropyLoss()
        self._optimizer_c_model = torch.optim.Adam(c_model.parameters(),
                                                   lr=RECEIVED_PARAMS['c_lr'],
                                                   weight_decay=0.0005)
        self._trainer_kwargs = trainer_kwargs

        eeg_dataset = dataset
        train_dataset, val_dataset = train_test_split(eeg_dataset)
        train_dataloader = DataLoader(train_dataset,
                                      batch_size=16,
                                      shuffle=True,
                                      drop_last=False)
        val_dataloader = DataLoader(val_dataset,
                                    batch_size=16,
                                    shuffle=False,
                                    drop_last=False)

        self._train_dataloader = train_dataloader
        self._val_dataloader = val_dataloader

    def _accuracy(self, input, target):  # pylint: disable=redefined-builtin
        _, predict = torch.max(input.data, 1)
        correct = predict.eq(target.data).cpu().sum().item()
        return correct / input.size(0)

    def training_step_c_model(self, batch, batch_idx):
        for p in self.c_model.parameters():
            p.requires_grad = True

        self._optimizer_c_model.zero_grad()

        x, y = batch
        x, y = x.cuda(), y.cuda()

        y_hat, x_feat = self.c_model(x)
        loss = self._loss_fn_ce(y_hat, y)

        loss.backward()
        self._optimizer_c_model.step()

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = self.validation_step_before_model(batch, batch_idx)
        y_hat, x_feat = self.c_model(x)
        return (y_hat.detach().cpu(), y.detach().cpu())

    def validation_step_before_model(self, batch, batch_idx):
        x, y = batch
        x, y = x.cuda(), y.cuda()
        return x, y

    def validation_epoch_end(self, outputs):
        # We might need dict metrics in future?
        y_hat, y = zip(*outputs)
        y_hat = torch.cat(y_hat, dim=0)
        y = torch.cat(y, dim=0)
        avg_acc = self._accuracy(y_hat, y)
        print("acc:", avg_acc)
        # logger.info('[VAL] Average ACC at epoch end is {}'.format(avg_acc))
        # return {'val_acc': avg_acc}

    def _validate(self, epoch_idx=-1):
        validation_outputs = []
        for i, batch in enumerate(self._val_dataloader):
            validation_outputs.append(self.validation_step(batch, i))
        return self.validation_epoch_end(validation_outputs)

    def _train(self, epoch_idx=-1):
        """
        单独显示每个 epoch 的训练进度条。
        """
        pbar = tqdm(total=len(self._train_dataloader), desc=f"[TRAIN] Epoch {epoch_idx}")
        for i, batch in enumerate(self._train_dataloader):
            loss_c_model = self.training_step_c_model(batch, i)
            pbar.update(1)
            # 更新进度条的后缀信息
            pbar.set_postfix(ordered_dict={'loss_c_model': f'{loss_c_model.item():.3f}'})
        pbar.close()

    def fit(self) -> None:
        """
        按照每个 epoch 单独创建训练和验证进度条。
        """
        for epoch_idx in range(self._trainer_kwargs['max_epochs']):
            self._train(epoch_idx + 1)
            self._validate(epoch_idx + 1)

        # logger.info('[VAL] Final ACC at experiment end is {}'.format(
        #     self._validate()['val_acc']))

    def save(self, param_path):
        torch.save({
            'c_model': self.c_model.state_dict(),
        }, param_path)


trainer = CTrainer(c_model, trainer_kwargs={'max_epochs': 80})
trainer.fit()
trainer.save('./parameters/' + 'cross_validation_backbone.pth')

NameError: name 'c_model' is not defined

In [25]:
c_model = SwinTransformer(patch_size=(4,3,3),
                            num_classes=2,
                            depths=(2, 4, 4),
                            num_heads=(2,3,4),
                            window_size=(3,3,3),
                            in_chans=1,
                            # drop_rate=0.2,
                            # attn_drop_rate=0.2,
                            # drop_path_rate=0.3,
                            visual=True
                            ) # T, W, H 同时缩小
g_model = Generator()

In [26]:
import os
import torch.nn.functional as F
import sys
class GCTrainer():
    def __init__(self, c_model, g_model, trainer_kwargs={'max_epochs': 10}):
        super().__init__()
        self.c_model = c_model.cuda()
        self.g_model = g_model.cuda()

        self._loss_fn_ce = nn.CrossEntropyLoss()
        self._loss_fn_mse = nn.MSELoss()
        self._optimizer_c_model = torch.optim.Adam(c_model.parameters(),
                                                   lr=RECEIVED_PARAMS['c_lr'],
                                                   weight_decay=0.0005)

        self._trainer_kwargs = trainer_kwargs

        eeg_dataset = dataset
        train_dataset, val_dataset = train_test_split(eeg_dataset)
        train_dataloader = DataLoader(train_dataset,
                                      batch_size=16,
                                      shuffle=True,
                                      drop_last=False)
        val_dataloader = DataLoader(val_dataset,
                                    batch_size=16,
                                    shuffle=False,
                                    drop_last=False)

        self._train_dataloader = train_dataloader
        self._val_dataloader = val_dataloader

    def _accuracy(self, input, target):  # pylint: disable=redefined-builtin
        _, predict = torch.max(input.data, 1)
        correct = predict.eq(target.data).cpu().sum().item()
        return correct / input.size(0)

    def training_step_c_model(self, batch, batch_idx):
        for p in self.c_model.parameters():
            p.requires_grad = True

        self._optimizer_c_model.zero_grad()

        x, y = batch
        x, y = x.cuda(), y.cuda()

        y_hat, x_feat = self.c_model(x)
        loss = self._loss_fn_ce(y_hat, y)

        aug_x, ratio = random_mask(x)
        aug_x = self.g_model(aug_x).detach()
        aug_y_hat, aug_x_feat = self.c_model(aug_x)

        loss += RECEIVED_PARAMS['weight_ssl'] * (
            (1 - ratio).squeeze() * F.mse_loss(
                x_feat, aug_x_feat, reduction='none').mean(dim=-1)).mean()

        loss.backward()
        self._optimizer_c_model.step()

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = self.validation_step_before_model(batch, batch_idx)
        y_hat, x_feat = self.c_model(x)
        return (y_hat.detach().cpu(), y.detach().cpu())

    def validation_step_before_model(self, batch, batch_idx):
        x, y = batch
        x, y = x.cuda(), y.cuda()
        return x, y

    def validation_epoch_end(self, outputs):
        # We might need dict metrics in future?
        y_hat, y = zip(*outputs)
        y_hat = torch.cat(y_hat, dim=0)
        y = torch.cat(y, dim=0)
        avg_acc = self._accuracy(y_hat, y)
        # logger.info('[VAL] Average ACC at epoch end is {}'.format(avg_acc))
        return {'val_acc': avg_acc}

    def _validate(self, epoch_idx=-1):
        validation_outputs = []
        for i, batch in enumerate(self._val_dataloader):
            validation_outputs.append(self.validation_step(batch, i))
        return self.validation_epoch_end(validation_outputs)

    def _train(self, epoch_idx=-1):
        """
        单独显示每个 epoch 的训练进度条。
        """
        pbar = tqdm(total=len(self._train_dataloader), desc=f"[TRAIN] Epoch {epoch_idx}")
        for i, batch in enumerate(self._train_dataloader):
            loss_c_model = self.training_step_c_model(batch, i)
            pbar.update(1)
            # 更新进度条的后缀信息
            pbar.set_postfix(ordered_dict={'loss_c_model': f'{loss_c_model.item():.3f}'})
        pbar.close()
        sys.stdout.flush()  # Ensure proper flushing after progress bar

    def fit(self) -> None:
        """
        按照每个 epoch 单独创建训练和验证进度条。
        """
        for epoch_idx in range(self._trainer_kwargs['max_epochs']):
            self._train(epoch_idx + 1)
                # 验证过程并获取验证结果
            val_metrics = self._validate(epoch_idx + 1)
            val_acc = val_metrics['val_acc']
            
            # 打印验证准确率
            # 或者每隔几个epoch打印一次
            if epoch_idx % 5 == 0:
                print(f"\n[EPOCH {epoch_idx + 1}] Validation Accuracy: {val_acc:.3f}")

    def save(self, param_path):
        torch.save({
            'c_model': trainer.c_model.state_dict(),
        }, param_path)

    def load(self):
        gan_model_state_dict = torch.load(
            './parameters/cross_validation_proposed_pretrain.pth')
        self.g_model.load_state_dict(gan_model_state_dict['g_model'])

        if os.path.exists('./parameters/cross_validation_backbone' + '.pth'):
            c_model_state_dict = torch.load(
                './parameters/cross_validation_backbone'  +
                '.pth')
            self.c_model.load_state_dict(c_model_state_dict['c_model'])


trainer = GCTrainer(c_model,
                  g_model,
                  trainer_kwargs={'max_epochs': 100})
trainer.load()
trainer.fit()
trainer.save('./parameters/' +  'cross_validation_finetune.pth')

[TRAIN] Epoch 1: 100%|██████████| 14/14 [00:01<00:00,  8.88it/s, loss_c_model=0.041]



[EPOCH 1] Validation Accuracy: 0.768


[TRAIN] Epoch 6: 100%|██████████| 14/14 [00:01<00:00,  8.88it/s, loss_c_model=0.030]



[EPOCH 6] Validation Accuracy: 0.750


[TRAIN] Epoch 11: 100%|██████████| 14/14 [00:01<00:00,  8.36it/s, loss_c_model=0.025]


...

[TRAIN] Epoch 91: 100%|██████████| 14/14 [00:01<00:00,  8.79it/s, loss_c_model=0.010]



[EPOCH 91] Validation Accuracy: 0.714


[TRAIN] Epoch 96: 100%|██████████| 14/14 [00:01<00:00,  8.83it/s, loss_c_model=0.007]



[EPOCH 96] Validation Accuracy: 0.768


[TRAIN] Epoch 100: 100%|██████████| 14/14 [00:01<00:00,  8.38it/s, loss_c_model=0.007]


In [21]:
import os
import torch.nn.functional as F
import sys
class GCTrainer(): 
    def __init__(self, train_dataloader, val_dataloader, c_model, g_model, trainer_kwargs={'max_epochs': 10}):
        super().__init__()
        self.c_model = c_model.cuda()
        self.g_model = g_model.cuda()

        self._loss_fn_ce = nn.CrossEntropyLoss()
        self._loss_fn_mse = nn.MSELoss()
        self._optimizer_c_model = torch.optim.Adam(c_model.parameters(),
                                                   lr=RECEIVED_PARAMS['c_lr'],
                                                   weight_decay=0.0005)

        self._trainer_kwargs = trainer_kwargs

        self._train_dataloader = train_dataloader
        self._val_dataloader = val_dataloader

    def _accuracy(self, input, target):  # pylint: disable=redefined-builtin
        _, predict = torch.max(input.data, 1)
        correct = predict.eq(target.data).cpu().sum().item()
        return correct / input.size(0)

    def training_step_c_model(self, batch, batch_idx):
        for p in self.c_model.parameters():
            p.requires_grad = True

        self._optimizer_c_model.zero_grad()

        x, y = batch
        x, y = x.cuda(), y.cuda()

        y_hat, x_feat = self.c_model(x)
        loss = self._loss_fn_ce(y_hat, y)

        aug_x, ratio = random_mask(x)
        aug_x = self.g_model(aug_x).detach()
        aug_y_hat, aug_x_feat = self.c_model(aug_x)

        loss += RECEIVED_PARAMS['weight_ssl'] * (
            (1 - ratio).squeeze() * F.mse_loss(
                x_feat, aug_x_feat, reduction='none').mean(dim=-1)).mean()

        loss.backward()
        self._optimizer_c_model.step()

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = self.validation_step_before_model(batch, batch_idx)
        y_hat, x_feat = self.c_model(x)
        return (y_hat.detach().cpu(), y.detach().cpu())

    def validation_step_before_model(self, batch, batch_idx):
        x, y = batch
        x, y = x.cuda(), y.cuda()
        return x, y

    def validation_epoch_end(self, outputs):
        # We might need dict metrics in future?
        y_hat, y = zip(*outputs)
        y_hat = torch.cat(y_hat, dim=0)
        y = torch.cat(y, dim=0)
        avg_acc = self._accuracy(y_hat, y)
        # logger.info('[VAL] Average ACC at epoch end is {}'.format(avg_acc))
        return {'val_acc': avg_acc}

    def _validate(self, epoch_idx=-1):
        validation_outputs = []
        for i, batch in enumerate(self._val_dataloader):
            validation_outputs.append(self.validation_step(batch, i))
        return self.validation_epoch_end(validation_outputs)

    def _train(self, epoch_idx=-1):
        """ 显示训练进度条 """
        pbar = tqdm(total=len(self._train_dataloader), desc=f"[TRAIN] Epoch {epoch_idx}")
        for i, batch in enumerate(self._train_dataloader):
            loss_c_model = self.training_step_c_model(batch, i)
            pbar.update(1)
            pbar.set_postfix(ordered_dict={'loss_c_model': f'{loss_c_model.item():.3f}'})
        pbar.close()
        sys.stdout.flush()

    def fit(self) -> None:
        """ 仅进行训练，不进行验证 """
        for epoch_idx in range(self._trainer_kwargs['max_epochs']):
            self._train(epoch_idx + 1)

    def test(self):
        """ 计算并返回整个验证集的结果 """
        self.c_model.eval()
        validation_outputs = []
        with torch.no_grad():
            for i, batch in enumerate(self._val_dataloader):
                x, y = batch
                x, y = x.cuda(), y.cuda()
                y_hat, _ = self.c_model(x)
                validation_outputs.append((y_hat.cpu(), y.cpu()))

        y_hat, y = zip(*validation_outputs)
        y_hat = torch.cat(y_hat, dim=0)
        y = torch.cat(y, dim=0)
        avg_acc = self._accuracy(y_hat, y)
        return {'val_acc': avg_acc}

    def save(self, param_path):
        torch.save({'c_model': self.c_model.state_dict()}, param_path)


    def load(self):
        gan_model_state_dict = torch.load(
            './parameters/cross_validation_proposed_pretrain.pth')
        self.g_model.load_state_dict(gan_model_state_dict['g_model'])

        if os.path.exists('./parameters/cross_validation_backbone' + '.pth'):
            c_model_state_dict = torch.load(
                './parameters/cross_validation_backbone'  +
                '.pth')
            self.c_model.load_state_dict(c_model_state_dict['c_model'])

In [22]:
import torch
import numpy as np
import torch.nn as nn

from tqdm import tqdm
from pathlib import Path
from torch import autograd
from sklearn import model_selection
from torch.utils.data import Dataset, Subset, DataLoader


RECEIVED_PARAMS = {
    "c_lr": 0.0001,
    "g_lr": 0.0001,
    "d_lr": 0.0001,
    "weight_gp": 1.0,
    "weight_decay": 0.005,
    "weight_ssl": 0.5
}
TRAIL_ID = 'cross_validation_proposed_pretrain'


class CFG:
    NUM_EPOCHS = 100
    NUM_CLASSES = 2
    BATCH_SIZE = 16
    TIMESTEP_NUM = 128


def gradient_penalty(model, real, fake):
    device = real.device
    real = real.data
    fake = fake.data
    alpha = torch.rand(real.size(0), *([1] * (len(real.shape) - 1))).to(device)
    inputs = alpha * real + ((1 - alpha) * fake)
    inputs.requires_grad_()
    outputs = model(inputs)

    gradient = autograd.grad(outputs=outputs,
                             inputs=inputs,
                             grad_outputs=torch.ones_like(outputs).to(device),
                             create_graph=True,
                             retain_graph=True,
                             only_inputs=True)[0]

    gradient = gradient.flatten(1)
    return ((gradient.norm(2, dim=1) - 1)**2).mean()


class Trainer():
    def __init__(self, g_model, d_model, train_dataloader, trainer_kwargs={'max_epochs': 10}):
        super().__init__()
        self.g_model = g_model.cuda()
        self.d_model = d_model.cuda()

        self._loss_fn_ce = nn.CrossEntropyLoss()
        self._loss_fn_mse = nn.MSELoss()
        self._optimizer_g_model = torch.optim.Adam(
            g_model.parameters(),
            lr=RECEIVED_PARAMS['g_lr'],
            weight_decay=RECEIVED_PARAMS['weight_decay'])
        self._optimizer_d_model = torch.optim.Adam(
            d_model.parameters(),
            lr=RECEIVED_PARAMS['d_lr'],
            weight_decay=RECEIVED_PARAMS['weight_decay'])

        self._trainer_kwargs = trainer_kwargs
        self._train_dataloader = train_dataloader

    def _accuracy(self, input, target):  # pylint: disable=redefined-builtin
        _, predict = torch.max(input.data, 1)
        correct = predict.eq(target.data).cpu().sum().item()
        return correct / input.size(0)

    def training_step_g_model(self, batch, batch_idx, augment_fn=random_mask):
        self._optimizer_g_model.zero_grad()

        for p in self.d_model.parameters():
            p.requires_grad = False

        x, y = batch
        x, y = x.cuda(), y.cuda()

        aug_x, ratio = random_mask(x)
        pred_x = self.g_model(aug_x)
        loss = -self.d_model(pred_x).mean() # 生成数据输入到d_model得到的数值越高，就越真。加个负号即最大化该数值

        loss.backward()
        self._optimizer_g_model.step()

        return loss

    def training_step_d_model(self, batch, batch_idx, augment_fn=random_mask):
        self._optimizer_d_model.zero_grad()

        for p in self.d_model.parameters():
            p.requires_grad = True

        x, y = batch
        x, y = x.cuda(), y.cuda()

        aug_x, ratio = random_mask(x)
        pred_x = self.g_model(aug_x).detach()

        fake_score = self.d_model(pred_x).mean()
        real_score = self.d_model(x).mean()
        loss = fake_score - real_score # 最大化生成数据和真实数据的差异
        # print("fake:",fake_score," real:",real_score)
        loss += RECEIVED_PARAMS['weight_gp'] * gradient_penalty(
            self.d_model, x, pred_x)

        if batch_idx % 5 == 0:
            loss.backward()
            self._optimizer_d_model.step()

        return loss

    def _train(self, epoch_idx=-1):
        """
        单独显示每个 epoch 的训练进度条，并动态更新 G 和 D 的损失。
        """
        pbar = tqdm(total=len(self._train_dataloader), desc=f"[TRAIN] Epoch {epoch_idx}")
        for i, batch in enumerate(self._train_dataloader):
            # 获取 D 模型的损失
            loss_d_model = self.training_step_d_model(batch, i)
            # 获取 G 模型的损失
            loss_g_model = self.training_step_g_model(batch, i)

            # 更新进度条
            pbar.update(1)
            pbar.set_postfix(
                ordered_dict={
                    'loss_g_model': f'{loss_g_model.item():.3f}',
                    'loss_d_model': f'{loss_d_model.item():.3f}'
                }
            )
        pbar.close()
        
    def fit(self) -> None:
        for i in range(self._trainer_kwargs['max_epochs']):
            self._train(i + 1)

    def save(self, param_path):
        torch.save(
            {
                'g_model': self.g_model.state_dict(),
                'd_model': self.d_model.state_dict()
            }, param_path)

In [23]:
class CTrainer():
    def __init__(self, c_model, train_dataloader, val_dataloader, trainer_kwargs={'max_epochs': 10}):
        super().__init__()
        self.c_model = c_model.cuda()

        self._loss_fn_ce = nn.CrossEntropyLoss()
        self._optimizer_c_model = torch.optim.Adam(c_model.parameters(),
                                                   lr=RECEIVED_PARAMS['c_lr'],
                                                   weight_decay=0.0005)
        self._trainer_kwargs = trainer_kwargs
        self._train_dataloader = train_dataloader
        self._val_dataloader = val_dataloader
        self.best_val_acc = 0.0  # 记录最佳验证准确率


    def _accuracy(self, input, target):  # pylint: disable=redefined-builtin
        _, predict = torch.max(input.data, 1)
        correct = predict.eq(target.data).cpu().sum().item()
        return correct / input.size(0)

    def training_step_c_model(self, batch, batch_idx):
        for p in self.c_model.parameters():
            p.requires_grad = True

        self._optimizer_c_model.zero_grad()

        x, y = batch
        x, y = x.cuda(), y.cuda()

        y_hat, x_feat = self.c_model(x)
        loss = self._loss_fn_ce(y_hat, y)

        loss.backward()
        self._optimizer_c_model.step()

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        x, y = x.cuda(), y.cuda()
        y_hat, _ = self.c_model(x)
        return y_hat.detach().cpu(), y.detach().cpu()

    def validation_epoch_end(self, outputs):
        # We might need dict metrics in future?
        y_hat, y = zip(*outputs)
        y_hat = torch.cat(y_hat, dim=0)
        y = torch.cat(y, dim=0)
        avg_acc = self._accuracy(y_hat, y)
        # print("acc:", avg_acc)
        return avg_acc  # 只返回准确率，不打印

    # def _validate(self, epoch_idx=-1):
    #     validation_outputs = []
    #     for i, batch in enumerate(self._val_dataloader):
    #         validation_outputs.append(self.validation_step(batch, i))
    #     return self.validation_epoch_end(validation_outputs)
    def _validate(self):
        validation_outputs = [self.validation_step(batch, i) for i, batch in enumerate(self._val_dataloader)]
        return self.validation_epoch_end(validation_outputs)

    def _train(self, epoch_idx=-1):
        pbar = tqdm(total=len(self._train_dataloader), desc=f"[TRAIN] Epoch {epoch_idx}")
        for i, batch in enumerate(self._train_dataloader):
            loss_c_model = self.training_step_c_model(batch, i)
            pbar.update(1)
            # 更新进度条的后缀信息
            pbar.set_postfix(ordered_dict={'loss_c_model': f'{loss_c_model.item():.3f}'})
        pbar.close()

    def fit(self) -> None:
        for epoch_idx in range(1, self._trainer_kwargs['max_epochs'] + 1):
            self._train(epoch_idx)
            val_acc = self._validate()

            # 记录最佳验证准确率
            if val_acc > self.best_val_acc:
                self.best_val_acc = val_acc
                self.save('./parameters/' +  'cross_validation_backbone.pth')
                print(f"New Best Validation Accuracy: {self.best_val_acc:.3f}, Model Saved!")
                
        print(f"Final Best Validation Accuracy: {self.best_val_acc:.3f}")

    def save(self, param_path):
        torch.save({
            'c_model': self.c_model.state_dict(),
        }, param_path)


In [8]:
from eegswintransformer import SwinTransformer
import torch

HYPERPARAMETERS = {
    "seed": 42,
    "batch_size": 16,
    "lr": 1e-4,
    "weight_decay": 1e-4,
    "num_epochs": 50,
}
from torcheeg.model_selection import KFoldPerSubjectGroupbyTrial
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torch.utils.data import DataLoader
from torcheeg.trainers import ClassifierTrainer

k_fold = KFoldPerSubjectGroupbyTrial(
    n_splits=4,
    shuffle=True,
    random_state=42)

test_metrics = []

for i, (training_dataset, test_dataset) in enumerate(k_fold.split(dataset)):
    # if i==0:
    #     continue
    training_loader = DataLoader(training_dataset,
                             batch_size=HYPERPARAMETERS['batch_size'],
                             shuffle=True)
    test_loader = DataLoader(test_dataset,
                             batch_size=HYPERPARAMETERS['batch_size'],
                             shuffle=False)
    # GAN 训练 generator & discriminator
    g_model = Generator()
    d_model = SwinTransformer(patch_size=(4,3,3),
                              num_classes=1,
                              depths=(2, 4, 4),
                              num_heads=(2,3,4),
                              window_size=(3,3,3),
                              in_chans=1,
                              drop_rate=0.2,
                              attn_drop_rate=0.2,
                              drop_path_rate=0.3
                              ) # T, W, H 同时缩小
    trainer1 = Trainer(g_model,
                      d_model,
                      training_loader,
                      trainer_kwargs={'max_epochs': 80})
    trainer1.fit()
    trainer1.save('./parameters/' + 'cross_validation_proposed_pretrain.pth')
    del g_model, d_model, trainer1
    torch.cuda.empty_cache()

    # classifier 预训练
    c_model = SwinTransformer(patch_size=(4,3,3),
                              num_classes=2,
                              depths=(2, 4, 4),
                              num_heads=(2,3,4),
                              window_size=(3,3,3),
                              in_chans=1,
                              drop_rate=0.2,
                              attn_drop_rate=0.2,
                              drop_path_rate=0.3,
                              visual=True
                              ) # T, W, H 同时缩小
    trainer2 = CTrainer(c_model, training_loader, test_loader, trainer_kwargs={'max_epochs': 60})
    trainer2.fit()
    # trainer2.save('./parameters/' + 'cross_validation_backbone.pth')
    del trainer2, c_model
    torch.cuda.empty_cache()

    # generator & classifier 训练
    g_model = Generator()
    c_model = SwinTransformer(patch_size=(4,3,3),
                              num_classes=2,
                              depths=(2, 4, 4),
                              num_heads=(2,3,4),
                              window_size=(3,3,3),
                              in_chans=1,
                              drop_rate=0.2,
                              attn_drop_rate=0.2,
                              drop_path_rate=0.3,
                              visual=True
                              ) # T, W, H 同时缩小 
    trainer3 = GCTrainer(training_loader,
                         test_loader,
                         c_model,
                         g_model,
                         trainer_kwargs={'max_epochs': 100})

    trainer3.load()
    trainer3.fit()
    with torch.no_grad():  # 避免测试时占用显存
        val_metrics = trainer3.test()
    test_metrics.append(val_metrics['val_acc'])
    print(f"Validation Accuracy: {val_metrics['val_acc']:.3f}")
    # trainer3.save('./parameters/' +  'cross_validation_finetune.pth')

    # 释放模型
    del trainer3, g_model, c_model
    torch.cuda.empty_cache()

[2025-03-06 23:35:07] INFO (torcheeg/MainThread) 📊 | Create the split of train and test set.
[2025-03-06 23:35:07] INFO (torcheeg/MainThread) 😊 | Please set split_path to .torcheeg/model_selection_1741275307926_qXNg1 for the next run, if you want to use the same setting for the experiment.
/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
[TRAIN] Epoch 1: 100%|██████████| 50/50 [00:02<00:00, 20.06it/s, loss_c_model=0.739]


New Best Validation Accuracy: 0.487, Model Saved!


[TRAIN] Epoch 3: 100%|██████████| 50/50 [00:02<00:00, 20.15it/s, loss_c_model=0.294]


New Best Validation Accuracy: 0.600, Model Saved!


[TRAIN] Epoch 4: 100%|██████████| 50/50 [00:02<00:00, 19.91it/s, loss_c_model=0.275]


...

New Best Validation Accuracy: 0.875, Model Saved!


[TRAIN] Epoch 60: 100%|██████████| 60/60 [00:02<00:00, 21.64it/s, loss_c_model=0.000]


Final Best Validation Accuracy: 0.875


[TRAIN] Epoch 100: 100%|██████████| 60/60 [00:04<00:00, 14.26it/s, loss_c_model=0.017]


Validation Accuracy: 0.850


In [25]:
from eegswintransformer import SwinTransformer
import torch

HYPERPARAMETERS = {
    "seed": 42,
    "batch_size": 8,
    "lr": 1e-4,
    "weight_decay": 1e-4,
    "num_epochs": 50,
}
from torcheeg.model_selection import KFoldPerSubjectGroupbyTrial
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torch.utils.data import DataLoader
from torcheeg.trainers import ClassifierTrainer

k_fold = KFoldPerSubjectGroupbyTrial(
    n_splits=4,
    shuffle=True,
    random_state=42)

test_metrics = []

for i, (training_dataset, test_dataset) in enumerate(k_fold.split(dataset)):
    # if i==0:
    #     continue
    training_loader = DataLoader(training_dataset,
                             batch_size=HYPERPARAMETERS['batch_size'],
                             shuffle=True)
    test_loader = DataLoader(test_dataset,
                             batch_size=HYPERPARAMETERS['batch_size'],
                             shuffle=False)
    # GAN 训练 generator & discriminator
    g_model = Generator()
    d_model = SwinTransformer(patch_size=(8,3,3),
                              num_classes=1,
                              depths=(2, 6, 4),
                              num_heads=(3,6,8),
                              window_size=(3,3,3),
                              in_chans=1,
                              drop_rate=0.2,
                              attn_drop_rate=0.2,
                              drop_path_rate=0.3
                              ) # T, W, H 同时缩小
    trainer1 = Trainer(g_model,
                      d_model,
                      training_loader,
                      trainer_kwargs={'max_epochs': 60})
    trainer1.fit()
    trainer1.save('./parameters/' + 'cross_validation_proposed_pretrain.pth')
    del g_model, d_model, trainer1
    torch.cuda.empty_cache()

    # classifier 预训练
    c_model = SwinTransformer(patch_size=(8,3,3),
                              num_classes=2,
                              depths=(2, 6, 4),
                              num_heads=(3,6,8),
                              window_size=(3,3,3),
                              in_chans=1,
                              drop_rate=0.2,
                              attn_drop_rate=0.2,
                              drop_path_rate=0.3,
                              visual=True
                              ) # T, W, H 同时缩小
    trainer2 = CTrainer(c_model, training_loader, test_loader, trainer_kwargs={'max_epochs': 40})
    trainer2.fit()
    # trainer2.save('./parameters/' + 'cross_validation_backbone.pth')
    del trainer2, c_model
    torch.cuda.empty_cache()

    # generator & classifier 训练
    g_model = Generator()
    c_model = SwinTransformer(patch_size=(8,3,3),
                              num_classes=2,
                              depths=(2, 6, 4),
                              num_heads=(3,6,8),
                              window_size=(3,3,3),
                              in_chans=1,
                              drop_rate=0.2,
                              attn_drop_rate=0.2,
                              drop_path_rate=0.3,
                              visual=True
                              ) # T, W, H 同时缩小 
    trainer3 = GCTrainer(training_loader,
                         test_loader,
                         c_model,
                         g_model,
                         trainer_kwargs={'max_epochs': 60})

    trainer3.load()
    trainer3.fit()
    with torch.no_grad():  # 避免测试时占用显存
        val_metrics = trainer3.test()
    test_metrics.append(val_metrics['val_acc'])
    print(f"Validation Accuracy: {val_metrics['val_acc']:.3f}")
    # trainer3.save('./parameters/' +  'cross_validation_finetune.pth')

    # 释放模型
    del trainer3, g_model, c_model
    torch.cuda.empty_cache()
     

[2025-03-09 20:59:53] INFO (torcheeg/MainThread) 📊 | Create the split of train and test set.
[2025-03-09 20:59:53] INFO (torcheeg/MainThread) 😊 | Please set split_path to .torcheeg/model_selection_1741525193378_LnjEz for the next run, if you want to use the same setting for the experiment.
[TRAIN] Epoch 1: 100%|██████████| 20/20 [00:01<00:00, 14.07it/s, loss_c_model=0.760]


New Best Validation Accuracy: 0.475, Model Saved!


[TRAIN] Epoch 2: 100%|██████████| 20/20 [00:01<00:00, 14.82it/s, loss_c_model=0.387]


New Best Validation Accuracy: 0.613, Model Saved!


[TRAIN] Epoch 4: 100%|██████████| 20/20 [00:01<00:00, 13.17it/s, loss_c_model=0.635]


...

[TRAIN] Epoch 40: 100%|██████████| 25/25 [00:01<00:00, 17.49it/s, loss_c_model=0.003]


Final Best Validation Accuracy: 0.875


[TRAIN] Epoch 60: 100%|██████████| 25/25 [00:02<00:00, 11.67it/s, loss_c_model=0.026]


Validation Accuracy: 0.875


FileNotFoundError: [Errno 2] No such file or directory: '.torcheeg/datasets_1741525001097_DQR1n/_record_0/eeg'

In [26]:
torch.cuda.empty_cache()


In [27]:
print(test_metrics)

[0.725, 0.8375, 0.875, 0.925, 0.6, 0.7, 0.925, 0.725, 0.65, 0.7, 0.675, 0.875, 0.6125, 0.8, 0.925, 0.9, 0.675, 0.725, 0.8, 0.8, 0.6625, 0.75, 0.75, 0.875, 0.6875, 0.5875, 0.85, 0.95, 0.675, 0.775, 0.9, 0.8, 0.6625, 0.625, 0.95, 0.825, 0.6375, 0.75, 0.825, 0.85, 0.6125, 0.7625, 0.85, 0.825, 0.6125, 0.7625, 0.825, 0.85, 0.65, 0.875, 0.9, 0.925, 0.6625, 0.825, 0.875, 0.875, 0.625, 0.7875, 0.9, 0.775, 0.6375, 0.775, 0.925, 0.85, 0.6, 0.8875, 0.875, 0.95, 0.675, 0.75, 0.625, 0.725, 0.6625, 0.8125, 0.925, 0.675, 0.575, 0.775, 0.85, 0.825, 0.6125, 0.7625, 0.825, 0.975, 0.7, 0.825, 0.925, 0.875, 0.575, 0.75, 0.875, 0.85, 0.625, 0.825, 0.875, 0.9, 0.6875, 0.7, 0.875, 0.9, 0.6625, 0.7, 0.975, 0.8, 0.65, 0.8875, 0.975, 0.925, 0.675, 0.85, 0.825, 0.775, 0.55, 0.7875, 0.825, 0.8, 0.5375, 0.575, 0.875]


In [28]:
for i in range(0, len(test_metrics), 4):
    print(f"{np.mean(test_metrics[i:i+4]):.3f}\t{np.std(test_metrics[i:i+4]):.3f}")


0.841	0.074
0.737	0.118
0.725	0.088
0.809	0.123
0.750	0.053
0.759	0.076
0.769	0.140
0.788	0.080
0.766	0.130
0.766	0.083
0.762	0.092
0.763	0.092
0.837	0.110
0.809	0.087
0.772	0.098
0.797	0.106
0.828	0.135
0.694	0.048
0.769	0.108
0.756	0.108
0.794	0.130
0.831	0.084
0.763	0.118
0.806	0.108
0.791	0.097
0.784	0.121
0.859	0.125
0.781	0.067
0.741	0.111
0.662	0.151


In [29]:
print(np.mean(test_metrics),np.std(test_metrics))

0.7779411764705881 0.1121165955314997


In [57]:
from torch.utils.data import Dataset, DataLoader
import torch
import random
from torcheeg.transforms import RandomNoise

class AugmentedEEGDataset(Dataset):
    def __init__(self, base_dataset, noise_prob=1):  
        self.base_dataset = base_dataset
        self.noise_transform = RandomNoise(p=noise_prob)

    def __len__(self):
        return len(self.base_dataset) * 2  # 原始数据 + 加噪数据

    def __getitem__(self, idx):
        original_idx = idx // 2  # 让索引在原数据和噪声数据之间切换
        eeg, label = self.base_dataset[original_idx]
        
        if idx % 2 == 1:  # 让一半数据加噪声
            eeg = self.noise_transform(eeg=eeg)['eeg']
        
        return eeg, label


In [13]:
import random

class AugmentedEEGDataset(Dataset):
    def __init__(self, base_dataset, augmentations=None):  
        """
        base_dataset: 原始 EEG 数据集
        augmentations: 数据增强方法的列表，每种方法都会增加数据
        """
        self.base_dataset = base_dataset
        self.augmentations = augmentations if augmentations else []
        self.num_augments = len(self.augmentations) + 1  # 原始数据 + N 种增强
        
    def __len__(self):
        return len(self.base_dataset) * self.num_augments  # 数据量扩大 num_augments 倍

    def __getitem__(self, idx):
        original_idx = idx // self.num_augments  # 映射到原数据
        eeg, label = self.base_dataset[original_idx]

        augment_idx = idx % self.num_augments
        if augment_idx > 0:  # 0 代表原始数据，其他索引使用增强
            eeg = self.augmentations[augment_idx - 1](eeg)  
            mask = (eeg.abs().sum(dim=1, keepdim=True) > 0).float()  # 计算 mask (batch, 1, 128, 9, 9)
            eeg = eeg*mask
        return eeg, label

        
# 定义数据增强方法
def add_gaussian_noise(eeg):
    noise = torch.randn_like(eeg) * 0.05  # 标准差 0.05 的高斯噪声
    mask = (eeg.abs().sum(dim=1, keepdim=True) > 0).float()  # 计算 mask (batch, 1, 128, 9, 9)
    return (eeg + noise)*mask

def time_shift(eeg):
    shift = random.randint(1, 10)  # 随机偏移 1~10 个时间点
    return torch.roll(eeg, shifts=shift, dims=1)

In [78]:
from eegswintransformer import SwinTransformer

HYPERPARAMETERS = {
    "seed": 42,
    "batch_size": 16,
    "lr": 1e-4,
    "weight_decay": 1e-4,
    "num_epochs": 50,
}
from torcheeg.model_selection import KFoldPerSubjectGroupbyTrial
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torch.utils.data import DataLoader
from torcheeg.trainers import ClassifierTrainer

k_fold = KFoldPerSubjectGroupbyTrial(
    n_splits=4,
    shuffle=True,
    split_path='.torcheeg/model_selection_1741180617687_7Mkqn',
    random_state=42)

training_metrics = []
test_metrics = []

for i, (training_dataset, test_dataset) in enumerate(k_fold.split(dataset)):
    # delete_folder_if_exists(target_folder_name='lightning_logs')
    model = SwinTransformer(patch_size=(4,3,3),
                              num_classes=2,
                              depths=(2, 4, 4),
                              num_heads=(2,3,4),
                              window_size=(3,3,3),
                              in_chans=1,
                              drop_rate=0.2,
                              attn_drop_rate=0.2,
                              drop_path_rate=0.3,
                            #   visual=True
                              ) # T, W, H 同时缩小
    # training_dataset = AugmentedEEGDataset(training_dataset)
    training_dataset = AugmentedEEGDataset(training_dataset,augmentations=[add_gaussian_noise, time_shift])
    # test_dataset = AugmentedEEGDataset(test_dataset)  # 测试集通常不需要增强，视情况而定

    training_loader = DataLoader(training_dataset,
                                batch_size=HYPERPARAMETERS['batch_size'],
                                shuffle=True)
    test_loader = DataLoader(test_dataset,
                            batch_size=HYPERPARAMETERS['batch_size'],
                            shuffle=False)
    trainer = ClassifierTrainer(model=model,
                                num_classes=2,
                                lr=HYPERPARAMETERS['lr'],
                                weight_decay=HYPERPARAMETERS['weight_decay'],
                                metrics=["accuracy"],
                                accelerator="gpu")
    # 提前停止回调
    early_stopping_callback = EarlyStopping(
        monitor='train_loss',
        patience=20,
        mode='min',
        verbose=True
    )
    trainer.fit(training_loader,
                test_loader,
                max_epochs=HYPERPARAMETERS['num_epochs'],
                callbacks=[early_stopping_callback],
                # enable_progress_bar=True,
                enable_model_summary=False,
                limit_val_batches=0.0)
    training_result = trainer.test(training_loader,
                                   enable_progress_bar=True,
                                   enable_model_summary=True)[0]
    test_result = trainer.test(test_loader,
                               enable_progress_bar=True,
                               enable_model_summary=True)[0]
    training_metrics.append(training_result["test_accuracy"])
    test_metrics.append(test_result["test_accuracy"])

[2025-03-05 21:41:32] INFO (torcheeg/MainThread) 📊 | Detected existing split of train and test set, use existing split from .torcheeg/model_selection_1741180617687_7Mkqn.
[2025-03-05 21:41:32] INFO (torcheeg/MainThread) 💡 | If the dataset is re-generated, you need to re-generate the split of the dataset instead of using the previous split.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0: 100%|██████████| 38/38 [00:02<00:00, 14.20it/s, loss=0.828, train_loss=0.573, train_accuracy=0.750]

Metric train_loss improved. New best score: 0.573


Epoch 0: 100%|██████████| 38/38 [00:02<00:00, 14.19it/s, loss=0.828, train_loss=0.573, train_accuracy=0.750]

/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_accuracy', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
[2025-03-05 21:41:36] INFO (torcheeg/MainThread) 
[Train] train_loss: 0.983 train_accuracy: 0.547 



...

Epoch 2: 100%|██████████| 45/45 [00:02<00:00, 15.66it/s, loss=0.499, train_loss=0.609, train_accuracy=0.688]

Metric train_loss improved by 0.363 >= min_delta = 0.0. New best score: 0.609


Epoch 2: 100%|██████████| 45/45 [00:02<00:00, 15.65it/s, loss=0.499, train_loss=0.609, train_accuracy=0.688]

[2025-03-05 21:46:38] INFO (torcheeg/MainThread) 
[Train] train_loss: 0.529 train_accuracy: 0.756 



Epoch 3:  16%|█▌        | 7/45 [00:00<00:02, 15.77it/s, loss=0.489, train_loss=0.343, train_accuracy=0.875] 

In [13]:
for i in range(0, len(test_metrics), 4):
    print(i,np.mean(training_metrics[i:i+4]), np.std(training_metrics[i:i+4]),np.mean(test_metrics[i:i+4]), np.std(test_metrics[i:i+4]))

NameError: name 'training_metrics' is not defined

In [80]:
for i, score in enumerate(test_metrics):
    print(f"{score:.3f}", end=' ')

0.637 0.688 0.587 0.675 